In [36]:
import pandas as pd

df=pd.read_csv(r"C:\Users\SRINATH\Desktop\data science\machine learing\ml project\data\raw\ultra_synthetic_200k.csv")
df["full_text"] = df["subject"] + " " + df["body"]
df["clean_text"] = df["full_text"].str.lower().str.replace(r"\s+", " ", regex=True)


In [37]:
df.shape

(200000, 12)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   subject     200000 non-null  object 
 1   body        200000 non-null  object 
 2   answer      200000 non-null  object 
 3   type        200000 non-null  object 
 4   queue       200000 non-null  object 
 5   priority    200000 non-null  object 
 6   language    200000 non-null  object 
 7   tag_1       200000 non-null  object 
 8   action      200000 non-null  object 
 9   ttr_hours   200000 non-null  float64
 10  full_text   200000 non-null  object 
 11  clean_text  200000 non-null  object 
dtypes: float64(1), object(11)
memory usage: 18.3+ MB


In [39]:
df.isna().sum( )

subject       0
body          0
answer        0
type          0
queue         0
priority      0
language      0
tag_1         0
action        0
ttr_hours     0
full_text     0
clean_text    0
dtype: int64

In [40]:
df.duplicated().sum()

np.int64(0)

In [41]:
from sklearn.model_selection import train_test_split

X = df["clean_text"]
y = df["queue"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features=30000,
    ngram_range=(1,2),
    min_df=2,
    sublinear_tf=True
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [43]:
from sklearn.svm import LinearSVC

model = LinearSVC(
    C=1.0,
    loss="squared_hinge",
    class_weight="balanced"
)

model.fit(X_train_vec, y_train)


,"penalty penalty: {'l1', 'l2'}, default='l2'Specifies the norm used in the penalization. The 'l2'penalty is the standard used in SVC. The 'l1' leads to ``coef_``vectors that are sparse.",'l2'
,"loss loss: {'hinge', 'squared_hinge'}, default='squared_hinge'Specifies the loss function. 'hinge' is the standard SVM loss(used e.g. by the SVC class) while 'squared_hinge' is thesquare of the hinge loss. The combination of ``penalty='l1'``and ``loss='hinge'`` is not supported.",'squared_hinge'
,"dual dual: ""auto"" or bool, default=""auto""Select the algorithm to either solve the dual or primaloptimization problem. Prefer dual=False when n_samples > n_features.`dual=""auto""` will choose the value of the parameter automatically,based on the values of `n_samples`, `n_features`, `loss`, `multi_class`and `penalty`. If `n_samples` < `n_features` and optimizer supportschosen `loss`, `multi_class` and `penalty`, then dual will be set to True,otherwise it will be set to False... versionchanged:: 1.3 The `""auto""` option is added in version 1.3 and will be the default in version 1.5.",'auto'
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"C C: float, default=1.0Regularization parameter. The strength of the regularization isinversely proportional to C. Must be strictly positive.For an intuitive visualization of the effects of scalingthe regularization parameter C, see:ref:`sphx_glr_auto_examples_svm_plot_svm_scale_c.py`.",1.0
,"multi_class multi_class: {'ovr', 'crammer_singer'}, default='ovr'Determines the multi-class strategy if `y` contains more thantwo classes.``""ovr""`` trains n_classes one-vs-rest classifiers, while``""crammer_singer""`` optimizes a joint objective over all classes.While `crammer_singer` is interesting from a theoretical perspectiveas it is consistent, it is seldom used in practice as it rarely leadsto better accuracy and is more expensive to compute.If ``""crammer_singer""`` is chosen, the options loss, penalty and dualwill be ignored.",'ovr'
,"fit_intercept fit_intercept: bool, default=TrueWhether or not to fit an intercept. If set to True, the feature vectoris extended to include an intercept term: `[x_1, ..., x_n, 1]`, where1 corresponds to the intercept. If set to False, no intercept will beused in calculations (i.e. data is expected to be already centered).",True
,"intercept_scaling intercept_scaling: float, default=1.0When `fit_intercept` is True, the instance vector x becomes ``[x_1,..., x_n, intercept_scaling]``, i.e. a ""synthetic"" feature with aconstant value equal to `intercept_scaling` is appended to the instancevector. The intercept becomes intercept_scaling * synthetic featureweight. Note that liblinear internally penalizes the intercept,treating it like any other term in the feature vector. To reduce theimpact of the regularization on the intercept, the `intercept_scaling`parameter can be set to a value greater than 1; the higher the value of`intercept_scaling`, the lower the impact of regularization on it.Then, the weights become `[w_x_1, ..., w_x_n,w_intercept*intercept_scaling]`, where `w_x_1, ..., w_x_n` representthe feature weights and the intercept weight is scaled by`intercept_scaling`. This scaling allows the intercept term to have adifferent regularization behavior compared to the other features.",1
,"class_weight class_weight: dict or 'balanced', default=NoneSet the parameter C of class i to ``class_weight[i]*C`` forSVC. If not given, all classes are supposed to haveweight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.",'balanced'
,"verbose verbose: int, default=0Enable verbose output. Note that this setting takes advantage of aper-process runtime setting in liblinear that, if enabled, may not workproperly in a multithreaded context.",0
,"random_state random_state: int, RandomState instance or None, default=NoneControls the pseud

In [44]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 1.0
                                 precision    recall  f1-score   support

           Billing and Payments       1.00      1.00      1.00      4000
               Customer Service       1.00      1.00      1.00      4000
                General Inquiry       1.00      1.00      1.00      4000
                Human Resources       1.00      1.00      1.00      4000
                     IT Support       1.00      1.00      1.00      4000
                Product Support       1.00      1.00      1.00      4000
          Returns and Exchanges       1.00      1.00      1.00      4000
            Sales and Pre-Sales       1.00      1.00      1.00      4000
Service Outages and Maintenance       1.00      1.00      1.00      4000
              Technical Support       1.00      1.00      1.00      4000

                       accuracy                           1.00     40000
                      macro avg       1.00      1.00      1.00     40000
                   weighted avg    

In [45]:
df.columns

Index(['subject', 'body', 'answer', 'type', 'queue', 'priority', 'language',
       'tag_1', 'action', 'ttr_hours', 'full_text', 'clean_text'],
      dtype='object')

In [46]:
X.iloc[123]

"rma status not updated can you pls check? thanks, michelle harmon. it's hitting user. hi team, i've a problem exchange request. it started today and i restarting but no luck not sure if this helps but sharing logs."

In [47]:
df.columns

Index(['subject', 'body', 'answer', 'type', 'queue', 'priority', 'language',
       'tag_1', 'action', 'ttr_hours', 'full_text', 'clean_text'],
      dtype='object')

In [48]:
text = """
Hello Support Team,

Since this morning, none of our employees can connect to the company VPN. 
The VPN client shows an authentication error and keeps disconnecting immediately. 
We already tried reinstalling the VPN client and restarting the network service, but nothing helped.

This is affecting remote work and needs quick attention.

Regards,
Kevin Turner
"""

pred = model.predict(vectorizer.transform([text]))
print(pred)



['IT Support']


Build the PRIORITY CLASSIFICATION MODEL

In [49]:
df.columns

Index(['subject', 'body', 'answer', 'type', 'queue', 'priority', 'language',
       'tag_1', 'action', 'ttr_hours', 'full_text', 'clean_text'],
      dtype='object')

In [50]:
X = df["clean_text"]
y = df["priority"]

In [51]:
X_train_p, X_test_p, y_train_priority, y_test_priority = train_test_split(
    df["clean_text"],
    df["priority"],
    test_size=0.2,
    random_state=42,
    stratify=df["priority"]
)


In [52]:
vectorizer = TfidfVectorizer(max_features=30000, ngram_range=(1,2))
X_train_vec = vectorizer.fit_transform(X_train_p)
X_test_vec = vectorizer.transform(X_test_p)


In [53]:
print(X_train_vec.shape)
print(len(y_train_priority))


(160000, 30000)
160000


In [54]:
X_test_vec

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2721192 stored elements and shape (40000, 30000)>

tuneing the hyperparameter 

In [55]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC

param_grid={"C":[0.1,0.1,3,5],
            "class_weight":[None,"balanced"],
            "loss":["squared_hinge"]}

priority_clf=LinearSVC()

grid=GridSearchCV(
    priority_clf,
    param_grid,
    cv=3,
    scoring="accuracy",
    n_jobs=-1,
    verbose=2)


In [56]:
grid.fit(X_train_vec, y_train_priority)


Fitting 3 folds for each of 8 candidates, totalling 24 fits


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",LinearSVC()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'C': [0.1, 0.1, ...], 'class_weight': [None, 'balanced'], 'loss': ['squared_hinge']}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",3
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also di

In [57]:
print("Best Params:", grid.best_params_)
print("Best Accuracy:", grid.best_score_)

Best Params: {'C': 0.1, 'class_weight': None, 'loss': 'squared_hinge'}
Best Accuracy: 0.9995812497265315


Retrain Final Priority Model with Best Params

In [58]:
from sklearn.svm import LinearSVC

priority_model = LinearSVC(
    C=0.1,
    class_weight="balanced",

    loss="squared_hinge"
)

priority_model.fit(X_train_vec, y_train_priority)










,"penalty penalty: {'l1', 'l2'}, default='l2'Specifies the norm used in the penalization. The 'l2'penalty is the standard used in SVC. The 'l1' leads to ``coef_``vectors that are sparse.",'l2'
,"loss loss: {'hinge', 'squared_hinge'}, default='squared_hinge'Specifies the loss function. 'hinge' is the standard SVM loss(used e.g. by the SVC class) while 'squared_hinge' is thesquare of the hinge loss. The combination of ``penalty='l1'``and ``loss='hinge'`` is not supported.",'squared_hinge'
,"dual dual: ""auto"" or bool, default=""auto""Select the algorithm to either solve the dual or primaloptimization problem. Prefer dual=False when n_samples > n_features.`dual=""auto""` will choose the value of the parameter automatically,based on the values of `n_samples`, `n_features`, `loss`, `multi_class`and `penalty`. If `n_samples` < `n_features` and optimizer supportschosen `loss`, `multi_class` and `penalty`, then dual will be set to True,otherwise it will be set to False... versionchanged:: 1.3 The `""auto""` option is added in version 1.3 and will be the default in version 1.5.",'auto'
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"C C: float, default=1.0Regularization parameter. The strength of the regularization isinversely proportional to C. Must be strictly positive.For an intuitive visualization of the effects of scalingthe regularization parameter C, see:ref:`sphx_glr_auto_examples_svm_plot_svm_scale_c.py`.",0.1
,"multi_class multi_class: {'ovr', 'crammer_singer'}, default='ovr'Determines the multi-class strategy if `y` contains more thantwo classes.``""ovr""`` trains n_classes one-vs-rest classifiers, while``""crammer_singer""`` optimizes a joint objective over all classes.While `crammer_singer` is interesting from a theoretical perspectiveas it is consistent, it is seldom used in practice as it rarely leadsto better accuracy and is more expensive to compute.If ``""crammer_singer""`` is chosen, the options loss, penalty and dualwill be ignored.",'ovr'
,"fit_intercept fit_intercept: bool, default=TrueWhether or not to fit an intercept. If set to True, the feature vectoris extended to include an intercept term: `[x_1, ..., x_n, 1]`, where1 corresponds to the intercept. If set to False, no intercept will beused in calculations (i.e. data is expected to be already centered).",True
,"intercept_scaling intercept_scaling: float, default=1.0When `fit_intercept` is True, the instance vector x becomes ``[x_1,..., x_n, intercept_scaling]``, i.e. a ""synthetic"" feature with aconstant value equal to `intercept_scaling` is appended to the instancevector. The intercept becomes intercept_scaling * synthetic featureweight. Note that liblinear internally penalizes the intercept,treating it like any other term in the feature vector. To reduce theimpact of the regularization on the intercept, the `intercept_scaling`parameter can be set to a value greater than 1; the higher the value of`intercept_scaling`, the lower the impact of regularization on it.Then, the weights become `[w_x_1, ..., w_x_n,w_intercept*intercept_scaling]`, where `w_x_1, ..., w_x_n` representthe feature weights and the intercept weight is scaled by`intercept_scaling`. This scaling allows the intercept term to have adifferent regularization behavior compared to the other features.",1
,"class_weight class_weight: dict or 'balanced', default=NoneSet the parameter C of class i to ``class_weight[i]*C`` forSVC. If not given, all classes are supposed to haveweight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.",'balanced'
,"verbose verbose: int, default=0Enable verbose output. Note that this setting takes advantage of aper-process runtime setting in liblinear that, if enabled, may not workproperly in a multithreaded context.",0
,"random_state random_state: int, RandomState instance or None, default=NoneControls the pseud

In [59]:
from sklearn.metrics import accuracy_score,classification_report

y_pred_priorirty=priority_model.predict(X_test_vec)

print("Final priority Accuracy",accuracy_score(y_test_priority,y_pred_priorirty))

print(classification_report(y_test_priority,y_pred_priorirty))

Final priority Accuracy 0.9996
              precision    recall  f1-score   support

        high       1.00      1.00      1.00     13354
         low       1.00      1.00      1.00     13299
      medium       1.00      1.00      1.00     13347

    accuracy                           1.00     40000
   macro avg       1.00      1.00      1.00     40000
weighted avg       1.00      1.00      1.00     40000



The action model training phase that we are gona build that 

In [60]:
X_action = df["clean_text"]
y_action = df["action"]


In [61]:
from sklearn.model_selection import train_test_split

X_train_a,X_test_a,y_train_a,y_test_a=train_test_split(X_action,
                                                       y_action,test_size=0.2,random_state=42,stratify=y_action)

TF-IDF Vectorization

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_a=TfidfVectorizer(max_features=30000,ngram_range=(1,2))


X_train_a_vec=vectorizer_a.fit_transform(X_train_a)
X_test_a_vec=vectorizer_a.transform(X_test_a)

— Train Action Model (LinearSVC)

In [63]:
from sklearn.svm import LinearSVC

action_model=LinearSVC(C=1.0,
                       loss="squared_hinge",
                       class_weight="balanced") 
action_model.fit(X_train_a_vec,y_train_a)

,"penalty penalty: {'l1', 'l2'}, default='l2'Specifies the norm used in the penalization. The 'l2'penalty is the standard used in SVC. The 'l1' leads to ``coef_``vectors that are sparse.",'l2'
,"loss loss: {'hinge', 'squared_hinge'}, default='squared_hinge'Specifies the loss function. 'hinge' is the standard SVM loss(used e.g. by the SVC class) while 'squared_hinge' is thesquare of the hinge loss. The combination of ``penalty='l1'``and ``loss='hinge'`` is not supported.",'squared_hinge'
,"dual dual: ""auto"" or bool, default=""auto""Select the algorithm to either solve the dual or primaloptimization problem. Prefer dual=False when n_samples > n_features.`dual=""auto""` will choose the value of the parameter automatically,based on the values of `n_samples`, `n_features`, `loss`, `multi_class`and `penalty`. If `n_samples` < `n_features` and optimizer supportschosen `loss`, `multi_class` and `penalty`, then dual will be set to True,otherwise it will be set to False... versionchanged:: 1.3 The `""auto""` option is added in version 1.3 and will be the default in version 1.5.",'auto'
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"C C: float, default=1.0Regularization parameter. The strength of the regularization isinversely proportional to C. Must be strictly positive.For an intuitive visualization of the effects of scalingthe regularization parameter C, see:ref:`sphx_glr_auto_examples_svm_plot_svm_scale_c.py`.",1.0
,"multi_class multi_class: {'ovr', 'crammer_singer'}, default='ovr'Determines the multi-class strategy if `y` contains more thantwo classes.``""ovr""`` trains n_classes one-vs-rest classifiers, while``""crammer_singer""`` optimizes a joint objective over all classes.While `crammer_singer` is interesting from a theoretical perspectiveas it is consistent, it is seldom used in practice as it rarely leadsto better accuracy and is more expensive to compute.If ``""crammer_singer""`` is chosen, the options loss, penalty and dualwill be ignored.",'ovr'
,"fit_intercept fit_intercept: bool, default=TrueWhether or not to fit an intercept. If set to True, the feature vectoris extended to include an intercept term: `[x_1, ..., x_n, 1]`, where1 corresponds to the intercept. If set to False, no intercept will beused in calculations (i.e. data is expected to be already centered).",True
,"intercept_scaling intercept_scaling: float, default=1.0When `fit_intercept` is True, the instance vector x becomes ``[x_1,..., x_n, intercept_scaling]``, i.e. a ""synthetic"" feature with aconstant value equal to `intercept_scaling` is appended to the instancevector. The intercept becomes intercept_scaling * synthetic featureweight. Note that liblinear internally penalizes the intercept,treating it like any other term in the feature vector. To reduce theimpact of the regularization on the intercept, the `intercept_scaling`parameter can be set to a value greater than 1; the higher the value of`intercept_scaling`, the lower the impact of regularization on it.Then, the weights become `[w_x_1, ..., w_x_n,w_intercept*intercept_scaling]`, where `w_x_1, ..., w_x_n` representthe feature weights and the intercept weight is scaled by`intercept_scaling`. This scaling allows the intercept term to have adifferent regularization behavior compared to the other features.",1
,"class_weight class_weight: dict or 'balanced', default=NoneSet the parameter C of class i to ``class_weight[i]*C`` forSVC. If not given, all classes are supposed to haveweight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.",'balanced'
,"verbose verbose: int, default=0Enable verbose output. Note that this setting takes advantage of aper-process runtime setting in liblinear that, if enabled, may not workproperly in a multithreaded context.",0
,"random_state random_state: int, RandomState instance or None, default=NoneControls the pseud

evaluaTE

In [64]:
from sklearn.metrics import accuracy_score,classification_report
y_pred_a=action_model.predict(X_test_a_vec)

In [65]:
y_pred_a

array(['escalate to support manager', 'verify internal configuration',
       'send pricing documentation', ..., 'validate refund status',
       'reset vpn credentials', 'send information packet'],
      shape=(40000,), dtype=object)

In [66]:
y_pred_a[1]

'verify internal configuration'

In [67]:
print("action model accuracy:",accuracy_score(y_test_a,y_pred_a))

action model accuracy: 0.999475


In [68]:
print("Action Model Accuracy:", accuracy_score(y_test_a, y_pred_a))
print(classification_report(y_test_a, y_pred_a))

Action Model Accuracy: 0.999475
                                      precision    recall  f1-score   support

                      approve return       1.00      1.00      1.00      1347
                   create bug report       1.00      1.00      1.00      1346
       escalate to incident response       1.00      1.00      1.00      1331
               escalate to logistics       1.00      1.00      1.00      1320
         escalate to support manager       1.00      1.00      1.00      1349
     forward to business development       1.00      1.00      1.00      1338
           initiate payment reversal       1.00      1.00      1.00      1328
             issue employment letter       1.00      1.00      1.00      1312
                   issue replacement       1.00      1.00      1.00      1333
   notify devops and restart service       1.00      1.00      1.00      1341
             perform system rollback       1.00      1.00      1.00       794
prioritize customer service res

In [69]:
# ========== PRIORITY MODEL (no saving) ==========
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

df = pd.read_csv(r"C:\Users\SRINATH\Desktop\data science\machine learing\ml project\data\raw\ultra_synthetic_200k.csv")

X = df["clean_text"].astype(str)
y = df["priority"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=30000)
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

clf = LinearSVC()
clf.fit(X_train_vec, y_train)

y_pred = clf.predict(X_test_vec)

print("Priority Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Priority Accuracy: 0.99955
              precision    recall  f1-score   support

        high       1.00      1.00      1.00     13354
         low       1.00      1.00      1.00     13299
      medium       1.00      1.00      1.00     13347

    accuracy                           1.00     40000
   macro avg       1.00      1.00      1.00     40000
weighted avg       1.00      1.00      1.00     40000



#2) ACTION MODEL

In [71]:
# ========== ACTION MODEL (no saving) ==========
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

df = pd.read_csv(r"C:\Users\SRINATH\Desktop\data science\machine learing\ml project\data\raw\ultra_synthetic_200k.csv")

X = df["clean_text"].astype(str)
y = df["action"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=35000)
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

clf = LinearSVC(class_weight="balanced")
clf.fit(X_train_vec, y_train)

y_pred = clf.predict(X_test_vec)

print("Action Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Action Accuracy: 0.999475
                                      precision    recall  f1-score   support

                      approve return       1.00      1.00      1.00      1347
                   create bug report       1.00      1.00      1.00      1346
       escalate to incident response       1.00      1.00      1.00      1331
               escalate to logistics       1.00      1.00      1.00      1320
         escalate to support manager       1.00      1.00      1.00      1349
     forward to business development       1.00      1.00      1.00      1338
           initiate payment reversal       1.00      1.00      1.00      1328
             issue employment letter       1.00      1.00      1.00      1312
                   issue replacement       1.00      1.00      1.00      1333
   notify devops and restart service       1.00      1.00      1.00      1341
             perform system rollback       1.00      1.00      1.00       794
prioritize customer service response 

3) TYPE MODEL

In [76]:
# ========== TYPE MODEL (no saving) ==========
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

df = pd.read_csv(r"C:\Users\SRINATH\Desktop\data science\machine learing\ml project\data\raw\ultra_synthetic_200k.csv")

X = df["clean_text"].astype(str)
y = df["type"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=25000)
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

clf = LinearSVC()
clf.fit(X_train_vec, y_train)

y_pred = clf.predict(X_test_vec)

print("Type Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Type Accuracy: 0.252025
              precision    recall  f1-score   support

      Change       0.25      0.25      0.25     10051
    Incident       0.25      0.26      0.25     10012
     Problem       0.26      0.25      0.25     10002
     Request       0.25      0.25      0.25      9935

    accuracy                           0.25     40000
   macro avg       0.25      0.25      0.25     40000
weighted avg       0.25      0.25      0.25     40000



TTR REGRESSION MODEL